# 300W_LP [Домашняя страница](http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/main.htm?utm_source=www.tensorflow.org&utm_medium=referral)
## Набор данных 300W-LP (Large Pose) основан на исходных изображениях 300W, <br /> расширенных алгоритмическими методами с 600 до 61 тыс. изображений. 

<img src="https://mlv4xkdrf2yq.i.optimole.com/cb:0l1_~a759/w:850/h:699/q:mauto/https://datagen.tech/app/uploads/2022/06/image1-2.png">

#### 300-W — это набор данных изображений лиц. Он содержит 300 изображений в помещении и 300 изображений на открытом воздухе с человеческими лицами, снятыми в дикой природе. Он охватывает разные лица, выражения лица, условия освещения, позы и размеры лица. Изображения были загружены с google.com с использованием поисковых запросов «вечеринка», «встреча», «протест» и «знаменитость». По сравнению с аналогичными наборами данных, 300W имеет более высокую долю частично закрытых изображений и необычные выражения лиц, такие как «удивление» и «крик». Изображения были аннотированы 68 точечными маркерами с использованием полуавтоматического метода. Размер изображений составляет от 48 КБ до 2 МБ.

In [1]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [ ]:
[300W-LP]: https://drive.google.com/file/d/0B7OEHD3T4eCkVGs0TkhUWFN6N1k/view?usp=sharing&resourcekey=0-WT5tO4TOCbNZY6r6z6WmOA

In [20]:
import scipy.io as sio
import os
import numpy as np

In [23]:
def get_ypr_from_mat(mat_path):
    # Get yaw, pitch, roll from .mat annotation.
    # They are in radians
    mat = sio.loadmat(mat_path)
    print(mat.keys())
    # [pitch yaw roll tdx tdy tdz scale_factor]
    pre_pose_params = mat['Pose_Para'][0]
    # Get [pitch, yaw, roll]
    pose_params = pre_pose_params[:3]
    return pose_params

In [24]:
get_ypr_from_mat("data/AFW/AFW_1051618982_1_0.mat")

dict_keys(['__header__', '__version__', '__globals__', 'pt2d', 'roi', 'Illum_Para', 'Color_Para', 'Tex_Para', 'Shape_Para', 'Exp_Para', 'Pose_Para'])


array([-0.08791836,  0.00418041,  0.0265818 ], dtype=float32)

In [ ]:
import scipy.io as sio
import os
import numpy as np

def get_ypr_from_mat(mat_path):
    # Get yaw, pitch, roll from .mat annotation.
    # They are in radians
    mat = sio.loadmat(mat_path)
    # [pitch yaw roll tdx tdy tdz scale_factor]
    pre_pose_params = mat['Pose_Para'][0]
    # Get [pitch, yaw, roll]
    pose_params = pre_pose_params[:3]
    return pose_params


dirr = "300W_LP"
f = open("300W_LP_val_annotation.txt", "w")
f.write("snp,x,y,z")
papki = os.listdir(dirr)
k=0
for i in range(len(papki)):
    file = os.listdir(dirr + "/" + papki[i])
    for j in range(len(file)):
        t = file[j].split(".")
        try:
            if t[1] == "mat":
                k+=1
                pose = get_ypr_from_mat(dirr + "/" + papki[i] + "/" + file[j])
                # And convert to degrees.
                pitch = pose[0] * 180 / np.pi
                yaw = pose[1] * 180 / np.pi
                roll = pose[2] * 180 / np.pi
                f.write(papki[i] + "/" + t[0] + '.jpg' + "," + str(yaw) + "," + str(pitch) + "," + str(roll))
        except:
            print(t)
print(k)

In [6]:
# coding=utf-8
# Copyright 2022 The TensorFlow Datasets Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""300W-LP Dataset."""

import os

import numpy as np
import tensorflow as tf
import tensorflow_datasets.public_api as tfds

_DATASET_URL = "https://drive.google.com/uc?export=download&id=0B7OEHD3T4eCkVGs0TkhUWFN6N1k"

_PROJECT_URL = "http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/main.htm"


class Builder(tfds.core.GeneratorBasedBuilder):
  """300W-LP dataset."""

  VERSION = tfds.core.Version("1.0.0")

  def _info(self):
    return self.dataset_info_from_configs(
        features=tfds.features.FeaturesDict({
            "image":
                tfds.features.Image(
                    shape=(450, 450, 3), encoding_format="jpeg"),
            "landmarks_origin":
                tfds.features.Tensor(shape=(68, 2), dtype=np.float32),
            "landmarks_2d":
                tfds.features.Tensor(shape=(68, 2), dtype=np.float32),
            "landmarks_3d":
                tfds.features.Tensor(shape=(68, 2), dtype=np.float32),
            "roi":
                tfds.features.Tensor(shape=(4,), dtype=np.float32),
            "illum_params":
                tfds.features.Tensor(shape=(10,), dtype=np.float32),
            "color_params":
                tfds.features.Tensor(shape=(7,), dtype=np.float32),
            "tex_params":
                tfds.features.Tensor(shape=(199,), dtype=np.float32),
            "shape_params":
                tfds.features.Tensor(shape=(199,), dtype=np.float32),
            "exp_params":
                tfds.features.Tensor(shape=(29,), dtype=np.float32),
            "pose_params":
                tfds.features.Tensor(shape=(7,), dtype=np.float32)
        }),
        homepage=_PROJECT_URL,
    )

  def _split_generators(self, dl_manager):
    """Returns SplitGenerators."""
    extracted_path = dl_manager.download_and_extract(_DATASET_URL)
    return [
        tfds.core.SplitGenerator(
            name=tfds.Split.TRAIN,
            gen_kwargs={
                "image_dir_path": os.path.join(extracted_path, "300W_LP"),
            }),
    ]

  def _generate_examples(self, image_dir_path):
    """Yields examples."""
    image_files = tf.io.gfile.glob(
        pattern=os.path.join(image_dir_path, "[!Code]*[!_Flip]/[!_]*.jpg"))
    label_files = [s.replace("jpg", "mat") for s in image_files]
    landmark_files = [
        s.replace("300W_LP", "300W_LP/landmarks").replace(".jpg", "_pts.mat")
        for s in image_files
    ]
    for image_file, label_file, landmark_file in zip(image_files, label_files,
                                                     landmark_files):
      with tf.io.gfile.GFile(label_file, "rb") as f:
        mat = tfds.core.lazy_imports.scipy.io.loadmat(f)
      pt2d_origin = mat["pt2d"].T
      pt2d_origin = (pt2d_origin / 450.0).astype(np.float32)
      roi = mat["roi"].reshape(4).astype(np.float32)
      illum_params = mat["Illum_Para"].reshape([-1]).astype(np.float32)
      color_params = mat["Color_Para"].reshape([-1]).astype(np.float32)
      tex_params = mat["Tex_Para"].reshape([-1]).astype(np.float32)
      shape_params = mat["Shape_Para"].reshape([-1]).astype(np.float32)
      exp_params = mat["Exp_Para"].reshape([-1]).astype(np.float32)
      pose_params = mat["Pose_Para"].reshape([-1]).astype(np.float32)
      with tf.io.gfile.GFile(landmark_file, "rb") as f:
        ldm_mat = tfds.core.lazy_imports.scipy.io.loadmat(f)
        pt2d = (ldm_mat["pts_2d"] / 450.0).astype(np.float32)
        pt3d = (ldm_mat["pts_3d"] / 450.0).astype(np.float32)
      record = {
          "image": image_file,
          "landmarks_origin": pt2d_origin,
          "landmarks_2d": pt2d,
          "landmarks_3d": pt3d,
          "roi": roi,
          "illum_params": illum_params,
          "color_params": color_params,
          "tex_params": tex_params,
          "shape_params": shape_params,
          "exp_params": exp_params,
          "pose_params": pose_params
      }
      yield os.path.basename(image_file), record

In [14]:
def generate_examples(image_dir_path):
    """Yields examples."""
    image_files = tf.io.gfile.glob(
        pattern=os.path.join(image_dir_path, "[!Code]*[!_Flip]/[!_]*.jpg"))
    label_files = [s.replace("jpg", "mat") for s in image_files]
    landmark_files = [
        s.replace("300W_LP", "300W_LP/landmarks").replace(".jpg", "_pts.mat")
        for s in image_files
    ]
    print(image_files,label_files,landmark_files)
    for image_file, label_file, landmark_file in zip(image_files, label_files,
                                                     landmark_files):
        with tf.io.gfile.GFile(label_file, "rb") as f:
            mat = tfds.core.lazy_imports.scipy.io.loadmat(f)
        pt2d_origin = mat["pt2d"].T
        pt2d_origin = (pt2d_origin / 450.0).astype(np.float32)
        roi = mat["roi"].reshape(4).astype(np.float32)
        illum_params = mat["Illum_Para"].reshape([-1]).astype(np.float32)
        color_params = mat["Color_Para"].reshape([-1]).astype(np.float32)
        tex_params = mat["Tex_Para"].reshape([-1]).astype(np.float32)
        shape_params = mat["Shape_Para"].reshape([-1]).astype(np.float32)
        exp_params = mat["Exp_Para"].reshape([-1]).astype(np.float32)
        pose_params = mat["Pose_Para"].reshape([-1]).astype(np.float32)
        with tf.io.gfile.GFile(landmark_file, "rb") as f:
            ldm_mat = tfds.core.lazy_imports.scipy.io.loadmat(f)
            pt2d = (ldm_mat["pts_2d"] / 450.0).astype(np.float32)
            pt3d = (ldm_mat["pts_3d"] / 450.0).astype(np.float32)
        record = {
          "image": image_file,
          "landmarks_origin": pt2d_origin,
          "landmarks_2d": pt2d,
          "landmarks_3d": pt3d,
          "roi": roi,
          "illum_params": illum_params,
          "color_params": color_params,
          "tex_params": tex_params,
          "shape_params": shape_params,
          "exp_params": exp_params,
          "pose_params": pose_params
        }
        return os.path.basename(image_file), record

In [17]:
print(generate_examples("data/AFW/AFW_1051618982_1_0.jpg"))

[] [] []
None
